In [1]:
import pandas as pd
url = 'https://github.com/mattharrison/datasets/raw/master'\
      '/data/dirtydevil.txt'
df = pd.read_csv(url, skiprows=lambda num: num <34 or num == 35,
                 sep='\t')
def to_denver_time(df_, time_col, tz_col):
    return (df_
       .assign(**{tz_col: df_[tz_col].replace('MDT', 'MST7MDT')})
       .groupby(tz_col)
       [time_col]
       .transform(lambda s: pd.to_datetime(s)
           .dt.tz_localize(s.name, ambiguous=True)
           .dt.tz_convert('America/Denver'))
    )

/home/matt/envs/menv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
def tweak_river(df_):
    return (df_
     .assign(datetime=to_denver_time(df_, 'datetime', 'tz_cd'))
     .rename(columns={'144166_00060': 'cfs',
                      '144167_00065': 'gage_height'})
     .set_index('datetime')
    )

In [3]:
dd = tweak_river(df)
dd

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
datetime,,,,,,,
2001-05-07 01:00:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:15:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:30:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:45:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,NaN
2001-05-07 02:00:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,NaN
...,...,...,...,...,...,...,...
2020-09-28 08:30:00-06:00,USGS,9333500,MDT,9.53,P,6.16,P
2020-09-28 08:45:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 09:00:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P


In [4]:
dd.to_csv('/tmp/dd.csv')  # doctest: +SKIP

In [5]:
print(dd.head(5).to_csv())

datetime,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
2001-05-07 01:00:00-06:00,USGS,9333500,MDT,71.0,A:[91],,
2001-05-07 01:15:00-06:00,USGS,9333500,MDT,71.0,A:[91],,
2001-05-07 01:30:00-06:00,USGS,9333500,MDT,71.0,A:[91],,
2001-05-07 01:45:00-06:00,USGS,9333500,MDT,70.0,A:[91],,
2001-05-07 02:00:00-06:00,USGS,9333500,MDT,70.0,A:[91],,



In [6]:
dd2 = pd.read_csv('/tmp/dd.csv', index_col='datetime')

/home/matt/envs/menv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
dd.to_excel('/tmp/dd.xlsx')

ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

In [8]:
(dd  # doctest: +SKIP
 .reset_index()
 .assign(datetime=lambda df_: df_.datetime.dt.tz_convert(tz=None))
 .set_index('datetime')
 .to_excel('/tmp/dd.xlsx')
)

In [9]:
writer = pd.ExcelWriter('/tmp/dd2.xlsx')
dd2 = (dd    
 .reset_index()
 .assign(datetime=lambda df_: df_.datetime.dt.tz_convert(tz=None))
 .set_index('datetime')
)
(dd2      # doctest: +SKIP
 .loc['2010':'2010-12-31']
 .to_excel(writer, sheet_name='2010')
)
(dd2     # doctest: +SKIP
 .loc['2011':'2011-12-31'] 
 .to_excel(writer, sheet_name='2011')
)
writer.save()      # doctest: +SKIP

In [10]:
dd.to_feather('/tmp/dd.fea')

ValueError: feather does not support serializing <class 'pandas.core.indexes.datetimes.DatetimeIndex'> for the index; you can .reset_index() to make the index into column(s)

In [11]:
(dd
 .reset_index()
 .to_feather('/tmp/dd.fea')
)

In [12]:
dd2 = pd.read_feather('/tmp/dd.fea')
dd2.set_index('datetime').equals(dd)

True

In [13]:
import sqlite3
con = sqlite3.connect('dd.db')
dd.to_sql('dd', con, if_exists='replace')  # doctest: +SKIP

In [14]:
import sqlalchemy as sa
eng = sa.create_engine('sqlite:///dd.db')
sa_con = eng.connect()
dd2 = pd.read_sql('dd', sa_con, index_col='datetime')  # doctest: +SKIP
dd2.equals(dd)  # doctest: +SKIP

False

In [15]:
dd2

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
datetime,,,,,,,
2001-05-07 01:00:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:15:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:30:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:45:00,USGS,9333500,MDT,70.00,A:[91],NaN,None
2001-05-07 02:00:00,USGS,9333500,MDT,70.00,A:[91],NaN,None
...,...,...,...,...,...,...,...
2020-09-28 08:30:00,USGS,9333500,MDT,9.53,P,6.16,P
2020-09-28 08:45:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 09:00:00,USGS,9333500,MDT,9.20,P,6.15,P


In [16]:
(dd2  # doctest: +SKIP
 .reset_index()
 .assign(datetime=lambda df_: df_.datetime
       .dt.tz_localize('America/Denver', ambiguous=False))
 .set_index('datetime')
 .equals(dd)
)

False

In [17]:
obj = dd.to_dict()

In [18]:
dd2 = pd.DataFrame.from_dict(obj)
dd.equals(dd2)

True

In [19]:
dd.to_json('/tmp/dd.json.gz')
dd2 = pd.read_json('/tmp/dd.json')
dd2

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
2001-05-07 07:00:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 07:15:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 07:30:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 07:45:00,USGS,9333500,MDT,70.00,A:[91],NaN,None
2001-05-07 08:00:00,USGS,9333500,MDT,70.00,A:[91],NaN,None
...,...,...,...,...,...,...,...
2020-09-28 14:30:00,USGS,9333500,MDT,9.53,P,6.16,P
2020-09-28 14:45:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 15:00:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 15:15:00,USGS,9333500,MDT,9.20,P,6.15,P


In [20]:
dd2.equals(dd)

False

In [21]:
dd3 = (dd2
 .reset_index()
 .rename(columns={'index':'datetime'})
 .assign(datetime=lambda df_: df_.datetime.dt.tz_localize(tz='UTC')
         .dt.tz_convert('America/Denver'))
 .set_index('datetime')
)

In [22]:
dd3

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
datetime,,,,,,,
2001-05-07 01:00:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:15:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:30:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:45:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,None
2001-05-07 02:00:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,None
...,...,...,...,...,...,...,...
2020-09-28 08:30:00-06:00,USGS,9333500,MDT,9.53,P,6.16,P
2020-09-28 08:45:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 09:00:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P


In [23]:
dd3.equals(dd)

False

In [24]:
dd3.round(3).equals(dd)

True

In [25]:
dd.head()

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
datetime,,,,,,,
2001-05-07 01:00:00-06:00,USGS,9333500,MDT,71.0,A:[91],NaN,NaN
2001-05-07 01:15:00-06:00,USGS,9333500,MDT,71.0,A:[91],NaN,NaN
2001-05-07 01:30:00-06:00,USGS,9333500,MDT,71.0,A:[91],NaN,NaN
2001-05-07 01:45:00-06:00,USGS,9333500,MDT,70.0,A:[91],NaN,NaN
2001-05-07 02:00:00-06:00,USGS,9333500,MDT,70.0,A:[91],NaN,NaN


In [26]:
dd.head().to_json()[:60]

'{"agency_cd":{"989218800000":"USGS","989219700000":"USGS","9'